In [1]:
from string import punctuation
from nltk.stem import WordNetLemmatizer


def process_text(text):
    # Function to remove stopwords
    def remove_stopwords(tokens):
        sw = stopwords.words("english")
        return [token for token in tokens if token not in sw]
    
    # Function to tokenize text
    def tokenize(text):
        return text.split()
    
    # Function to remove punctuation
    def remove_punctuation(text):
        return "".join([ch if ch not in punctuation else " " for ch in text])
    
    # Function to convert tokens to lowercase
    def lowercase_tokens(tokens):
        return [token.lower() for token in tokens]
    
    # Function to remove numerical tokens
    def remove_numerical_tokens(tokens):
        return [token for token in tokens if not token.isdigit()]
    
    # Function to lemmatize tokens
    def lemmatize_tokens(tokens):
        lemmatizer = WordNetLemmatizer()
        return [lemmatizer.lemmatize(token) for token in tokens]
    
    # Function to remove duplicate tokens
    def remove_duplicate_tokens(tokens):
        return list(OrderedDict.fromkeys(tokens))
    
    # Function to strip extra spaces
    def strip_extra_spaces(text):
        return " ".join(text.split())
    
    # Pipeline of text processing steps
    pipeline = [
        remove_punctuation,
        strip_extra_spaces,
        tokenize,
        lowercase_tokens,
        remove_numerical_tokens,
        lemmatize_tokens,
        remove_stopwords,
        remove_duplicate_tokens
    ]
    
    # Apply the pipeline to the text
    processed_text = text
    for function in pipeline:
        processed_text = function(processed_text)
    
    return processed_text